In [20]:
# let's get set up, make sure everything is installed and imported 

import sys
!{sys.executable} -m pip install joblib
!{sys.executable} -m pip install re
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install collections
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install time


import re
import requests
import requests.exceptions
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd
import time   


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [32]:
# insert a string of text that includes the URLs you want to search for email addresses

text = '''TEXT INCLUDING www.URLS.com HERE'''

# for example, this should look like: 

# text = '''SUMMARY OF CITY & TOWN INFORMATION
# APACHE JUNCTION, CITY OF
# 300 East Superstition Boulevard
# Apache Junction, AZ 85119
# Phone: (480) 982-8002
# Fax: 0
# www.apachejunctionaz.gov
# AVONDALE, CITY OF
# 11465 West Civic Center Drive
# Avondale, AZ 85323
# Phone: (623) 333-1000
# Fax: (623) 333-0101
# www.avondaleaz.gov
# BENSON, CITY OF
# P.O. Box 2223
# Benson, AZ 85602
# Phone: (520) 586-2245
# Fax: (520) 586-3375
# www.cityofbenson.com'''

In [33]:
URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

In [34]:
urls = list(re.findall(URL_REGEX, text))

print("count of URL(s) you're about to scrape: "+str(len(urls)))


count of URL(s) you're about to scrape: 3


In [39]:
# assign integer for the depth of how many pages you want to scrape on each URL

limit_urls_to = 100

# assign an integer for how many emails you want to reach before moving to the next URL

limit_emails_to = 20

# the code will move to the next URL when it hits the first of the above thresholds 


In [40]:
# much of this is lifted from https://pythoncircle.com/post/217/python-script-2-crawling-all-emails-from-a-website/

def scrape_sites(target_url):

    start = time.time()  

    starting_url = 'http://'+target_url

    # a queue of urls to be crawled
    unprocessed_urls = deque([starting_url])

    # set of already crawled urls
    processed_urls = set()

    # a set of fetched emails
    emails = set()

    # setting the limit of how many pages you're going to scrape per each base URL, before moving on
    url_limit = limit_urls_to

    # setting the limit of how many emails you're going to gather per each base URL, before moving on
    email_floor = limit_emails_to

    # process urls one by one from unprocessed_url queue until queue is empty
    while len(unprocessed_urls) and (sum((itm.count(target_url) for itm in processed_urls)))<url_limit and len(emails)<=email_floor:

        # move next url from the queue to the set of processed urls
        url = unprocessed_urls.popleft()
        processed_urls.add(url)

        # extract base url to resolve relative links
        parts = urlsplit(url)
        base_url = "{0.scheme}://{0.netloc}".format(parts)
        path = url[:url.rfind('/')+1] if '/' in parts.path else url

        # get url's content
        elapsed = time.time() - start
        print("Crawling URL %s" % url)
        print(str(sum((itm.count(target_url) for itm in processed_urls)))+ " URLs crawled in " + target_url + " in " + str(elapsed) +" seconds ")
        print("So that's " + (str(elapsed/(sum((itm.count(target_url) for itm in processed_urls))))) + " seconds per page")
        print("and " + str(len(emails)) + " emails")
        print("")

        try:
            response = requests.get(url)
        except:
            # ignore pages with errors and continue with next url
            pass
        

        # extract all email addresses and add them into the resulting set
        # You may edit the regular expression as per your requirement
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
        emails.update(new_emails)
        #print(emails)

        # create a beutiful soup for the html document
        soup = BeautifulSoup(response.text, 'lxml')

        # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
        for anchor in soup.find_all("a"):

            # extract link url from the anchor
            link = anchor.attrs["href"] if "href" in anchor.attrs else ''

            # resolve relative links (starting with /)
            if link.startswith('/'):
                link = base_url + link
            elif not link.startswith('http'):
                link = path + link

            # add the new url to the queue if it was not in unprocessed list nor in processed list yet
            if not link in unprocessed_urls and not link in processed_urls and url in link: 
                unprocessed_urls.append(link)

        output = pd.DataFrame(list(emails))
    
    return output.to_csv(target_url+'_emails_.csv')
    
                

In [41]:
# spin up your computer's full horsepower to speed up this process 

import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  12


In [42]:
# deploy all your processors to scraping URLs

from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

try: 
    Parallel(n_jobs=num_cores)(delayed(scrape_sites)(target_url) for target_url in urls)

except: 
    pass 

